In [1]:
import pickle
import cobra
import lftc
import igraph
import pandas as pd

In [2]:
results = pickle.load(open('test_data/results_tca_only_longrun.p', 'rb'))[39][0]
len(results)

127

In [3]:
coreReactionNames = {
    'GLCpts',
    'PGI',
    'PFK',
    'FBA',
    'GAPD',
    'PGK',
    'PGM',
    'ENO',
    'PYK',
    'DHAPT',
    'PDH',
    'CS',
    'ACONT',
    'ICL',
    'ICDHyr',
    'AKGDH',
    'SUCOAS',
    'SUCD1i',
    'FUM',
    'MDH',
}


In [4]:
model = cobra.io.read_sbml_model('test_data/EciJR904TKs_sbml3.xml')
feed='EX_glc_e_'

In [5]:
len(results)/len(model.reactions)

0.11792014856081709

In [6]:
sortedMetabolites = pd.DataFrame([(m.id, len(m.reactions)) for m in model.metabolites]).sort_values(1,ascending=False)
set(sortedMetabolites[0:0][0]).difference(lftc.currencyMetabolites)

set()

In [7]:
len(model.reactions)

1077

In [8]:
modelGraph = igraph.Graph(directed=False)
currencyMetabolites=lftc.currencyMetabolites
modelGraph.add_vertex(name=feed)
newCoreReactionNames = set()
unexploredReactionNames = {feed} # this starts with just the feed name
while len(unexploredReactionNames) > 0:
    reactionName = unexploredReactionNames.pop()
    newCoreReactionNames.update([reactionName])
    # modelGraph.add_vertex(name=reactionName)
    reaction = model.reactions.get_by_id(reactionName)
    connectedMetabolites = {m.id for m in reaction.reactants}
    connectedMetabolites.update({m.id for m in reaction.products})
    connectedMetabolites = connectedMetabolites.difference(currencyMetabolites)
    sourceNode = modelGraph.vs["name"].index(reactionName)
    
    for metaboliteName in connectedMetabolites:
        metabolite = model.metabolites.get_by_id(metaboliteName)
        connectedReactionNames = {r.id for r in metabolite.reactions}
        # connectedReactionNames = connectedReactionNames.intersection(coreReactionNames)
        newReactionsThisLoop = connectedReactionNames.difference(newCoreReactionNames)
        unexploredReactionNames.update(newReactionsThisLoop)
        newCoreReactionNames.update(newReactionsThisLoop)
        for newReactionName in connectedReactionNames:
            # print("connected to", reactionName, len(newReactionName))
            if not newReactionName in modelGraph.vs["name"]:
                modelGraph.add_vertex(name=newReactionName)
            
            targetNode = modelGraph.vs["name"].index(newReactionName)
            if not modelGraph.are_connected(sourceNode, targetNode):
                modelGraph.add_edge(source=sourceNode, target=targetNode)

In [9]:
print(modelGraph)

IGRAPH UN-- 1028 8212 --
+ attr: name (v)
+ edges (vertex names):
     EX_glc_e_ -- EX_glc_e_, EX_glc_e_, GLCt2, TREHe, GLCDe, GLCpts
         GLCt2 -- EX_glc_e_, GLCt2, GLCt2, TREHe, GLCDe, GLCpts, MLTG3,
AMALT3, MLTG5, AMALT2, AMALT1, TRE6PH, TREH, LACZ, G1PP, MLTG2, MLTG1, MLTG4,
XYLI2i, GALS3, AMALT4, HEX1
         TREHe -- EX_glc_e_, GLCt2, TREHe, TREHe, GLCDe, GLCpts, DMSOR1e,
NADDPe, H2Ot, EX_h2o_e_, DMSOR2e, TMAOR1e, TMAOR2e, TREpts, EX_tre_e_
         GLCDe -- EX_glc_e_, GLCt2, TREHe, GLCDe, GLCDe, GLCpts, EX_glcn_e_,
GLCNt2r, DMSOR1e, NADDPe, H2Ot, EX_h2o_e_, DMSOR2e, TMAOR1e, TMAOR2e
        GLCpts -- EX_glc_e_, GLCt2, TREHe, GLCDe, GLCpts, GLCpts, TREpts,
TRE6PH, HEX1, PGMT, TRE6PS, AB6PGH, G6PDH2r, FFSD, G6Pt6_2, PGI, ACNML,
L_LACD3, SBTpts, SERD_L, ICHORT, ALATA_L2, ALATA_D2, ANS, DAAD, ACGApts,
SERD_D, SHCHCS2, ADCL, CHRPL, GALTpts, ME1, EDA, FRUpts2, MALTpts, DHDPS,
PYRt2r, L_LACD2, MANpts, VPAMT, POX, DDPGALA, MCITL2, HOPNTAL, PPS, ACHBS,
PFL, GAMpts, LDH_D, PDH, PYK, 

In [10]:
modelGraph.is_connected(mode=igraph.WEAK)

True

In [11]:
for reactionName in results:
    modelGraph.vs[modelGraph.vs["name"].index(reactionName)]['class'] = 'newcore'

for reactionName in coreReactionNames:
    modelGraph.vs[modelGraph.vs["name"].index(reactionName)]['class'] = 'oldcore'
    


In [12]:
modelGraph.save('test_data/graph.graphml', 'graphml')